# Convert data to CLAHE
This notebook shows how to prepare the data for inference
* The data download from ESRF are originally jp2
* Before inference, we will need to convert to .tif volume followed by a 3D CLAHE

In [1]:
import sys
sys.path.append('..')
import os
import glob
import natsort
import numpy as np
import skimage.io as skio
from segmentation.preprocessing import preprocessor


In [5]:
image_16bit_jp2_dir = '/hdd/yang/projects/glomeruli_segmentation/data/LADAF-2021-17_left/5.2um_LADAF-2021-17_kidney-left_VOI-01.1_pag-0.26_0.11_jp2_'
volume_save_dir = '/hdd/yang/projects/glomeruli_segmentation/data/LADAF-2021-17_left/5.2um_LADAF-2021-17_kidney-left_VOI-01.1_pag-0.26_0.11_16bit_volume/'
if not os.path.exists(volume_save_dir):
    os.makedirs(volume_save_dir)
# check folder size

def get_size(start_path = '.'):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            # skip if it is symbolic link
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)

    return total_size

print(get_size(image_16bit_jp2_dir)/1e9, 'GB')

12.033088989 GB


# Divide big CLAHE patches for inference

In [3]:
preprocessor.devide_subvolumes(
    big_vol_path='/hdd/yang/projects/glomeruli_segmentation/data/LADAF-2021-17-left/5.2um_LADAF-2021-17_kidney-left_VOI-01.1_pag-0.26_0.11_clahe_/LADAF-2021-17-left-voi01.1-5.2um-clahe_0000.tif',
    segments=2,
    overlap_depth_in_pixels=64
)

Big volume shape: (6475, 1928, 1928)
Segment size: 3237


 50%|█████     | 1/2 [00:05<00:05,  5.64s/it]

Subvolume saved at: /hdd/yang/projects/glomeruli_segmentation/data/LADAF-2021-17-left/5.2um_LADAF-2021-17_kidney-left_VOI-01.1_pag-0.26_0.11_clahe_/LADAF-2021-17-left-voi01.1-5.2um-clahe_0000_subvol_0_3237.tif
Subvolume shape: (3237, 1928, 1928)


100%|██████████| 2/2 [00:11<00:00,  5.71s/it]

Subvolume saved at: /hdd/yang/projects/glomeruli_segmentation/data/LADAF-2021-17-left/5.2um_LADAF-2021-17_kidney-left_VOI-01.1_pag-0.26_0.11_clahe_/LADAF-2021-17-left-voi01.1-5.2um-clahe_0000_subvol_3173_6474.tif
Subvolume shape: (3301, 1928, 1928)
Subvolume division completed.


# Normalise Data for registration

In [1]:
# Normalise data for registration
import sys
sys.path.append('..')
from segmentation.preprocessing import preprocessor

input_dir = '/hdd/yang/projects/glomeruli_segmentation/data/LADAF-2021-17-left/5.2um_voi01/5.2um_LADAF-2021-17_kidney-left_VOI-01.1_pag-0.26_0.11_jp2_/'
output_dir = '/hdd/yang/projects/glomeruli_segmentation/data/LADAF-2021-17-left/5.2um_voi01/5.2um_LADAF-2021-17_kidney-left_VOI-01.1_pag-0.26_0.11_tif_normalised/'

normaliser = preprocessor.hipctNormaliser(
    input_dir, 
    output_dir, 
    convert_to_8_bit=True, 
    masked=True, 
    threshold=0.01)

normaliser.normalise_multiprocessing()

Number of files: 6475 
 File type: uint16 
 Converting to 8 bit: True 
 Masked: True
Multi-processing with 16 CPUs...


KeyboardInterrupt: 